# Motivational Example

Let start with some simple example; we will repeat the experiment from the chapter 6 of the book (codes avalaible [here](https://github.com/ftheberge/GraphMiningNotebooks/blob/master/Python_Notebooks/Chapter_6.ipynb) and [here](https://github.com/ftheberge/GraphMiningNotebooks/blob/master/Julia_Notebooks/Chapter_6.ipynb)): 

In [1]:
import igraph as ig
import numpy as np
import scipy.sparse.linalg as lg
datadir = 'C:/users/p/Datasets/'

In [2]:
## Hope embedding with various similarity functions## Hope embedding with various similarity functions
def Hope(g, sim='katz', dim=2, verbose=False, beta=.01, alpha=.5):
    ## For undirected graphs, embedding as source and target are identical
    if g.is_directed() == False:
        dim = dim*2
    A = np.array(g.get_adjacency().data)
    beta = beta
    alpha = alpha
    n = g.vcount()
    ## Katz
    if sim == 'katz':
        M_g = np.eye(n) - beta * A
        M_l = beta * A
    ## Adamic-Adar
    if sim == 'aa':
        M_g = np.eye(n)
        ## fix bug 1/x and take log();
        D = np.diag([1/np.log(x) if x>1 else 0 for x in g.degree()]) 
        # D = np.diag([1/np.log(max(2,x)) for x in g.degree()]) 
        M_l = np.dot(np.dot(A,D),A)
        np.fill_diagonal(M_l,0)
    ## Common neighbors
    if sim == 'cn':
        M_g = np.eye(n)
        M_l = np.dot(A,A)
    ## presonalized page rank
    if sim == 'ppr':
        P = []
        for i in range(n):
            s = np.sum(A[i])
            if s>0:
                P.append([x/s for x in A[i]])
            else:
                P.append([1/n for x in A[i]])
        P = np.transpose(np.array(P)) ## fix bug - take transpose
        M_g = np.eye(n)-alpha*P
        M_l = (1-alpha)*np.eye(n)
    S = np.dot(np.linalg.inv(M_g), M_l)
    u, s, vt = lg.svds(S, k=dim // 2)
    X1 = np.dot(u, np.diag(np.sqrt(s)))
    X2 = np.dot(vt.T, np.diag(np.sqrt(s)))
    X = np.concatenate((X1, X2), axis=1)
    p_d_p_t = np.dot(u, np.dot(np.diag(s), vt))
    eig_err = np.linalg.norm(p_d_p_t - S)
    if verbose:
        print('SVD error (low rank): %f' % eig_err)
    ## undirected graphs have identical source and target embeddings
    if g.is_directed() == False:
        d = dim//2
        return X[:,:d]
    else:
        return X

def Hope(g, sim='katz', dim=2, verbose=False, beta=.01, alpha=.5):
    ## For undirected graphs, embedding as source and target are identical
    if g.is_directed() == False:
        dim = dim*2
    A = np.array(g.get_adjacency().data)
    beta = beta
    alpha = alpha
    n = g.vcount()
    ## Katz
    if sim == 'katz':
        M_g = np.eye(n) - beta * A
        M_l = beta * A
    ## Adamic-Adar
    if sim == 'aa':
        M_g = np.eye(n)
        ## fix bug 1/x and take log();
        D = np.diag([1/np.log(x) if x>1 else 0 for x in g.degree()]) 
        # D = np.diag([1/np.log(max(2,x)) for x in g.degree()]) 
        M_l = np.dot(np.dot(A,D),A)
        np.fill_diagonal(M_l,0)
    ## Common neighbors
    if sim == 'cn':
        M_g = np.eye(n)
        M_l = np.dot(A,A)
    ## presonalized page rank
    if sim == 'ppr':
        P = []
        for i in range(n):
            s = np.sum(A[i])
            if s>0:
                P.append([x/s for x in A[i]])
            else:
                P.append([1/n for x in A[i]])
        P = np.transpose(np.array(P)) ## fix bug - take transpose
        M_g = np.eye(n)-alpha*P
        M_l = (1-alpha)*np.eye(n)
    S = np.dot(np.linalg.inv(M_g), M_l)
    u, s, vt = lg.svds(S, k=dim // 2)
    X1 = np.dot(u, np.diag(np.sqrt(s)))
    X2 = np.dot(vt.T, np.diag(np.sqrt(s)))
    X = np.concatenate((X1, X2), axis=1)
    p_d_p_t = np.dot(u, np.dot(np.diag(s), vt))
    eig_err = np.linalg.norm(p_d_p_t - S)
    if verbose:
        print('SVD error (low rank): %f' % eig_err)
    ## undirected graphs have identical source and target embeddings
    if g.is_directed() == False:
        d = dim//2
        return X[:,:d]
    else:
        return X


In [4]:
ABCD = ig.Graph.Read_Ncol(datadir+'ABCD/abcd_1000.dat',directed=False)

In [6]:
%time Hope(ABCD,"ppr", dim = 16);

Wall time: 1.66 s


array([[-0.02223087, -0.071124  ,  0.00850494, ..., -0.06271747,
         0.02352131,  0.04046968],
       [ 0.02621353,  0.00274195, -0.01033648, ...,  0.00911985,
        -0.00938255,  0.02893754],
       [-0.06561862,  0.08118835, -0.00949355, ..., -0.09410544,
         0.02422925,  0.10367872],
       ...,
       [-0.01904609, -0.00536347, -0.02059628, ...,  0.01611993,
         0.01341896,  0.02385654],
       [-0.01502635,  0.01431598, -0.04754913, ..., -0.00729053,
         0.02757331,  0.02194937],
       [-0.01034281, -0.01610467, -0.05423679, ...,  0.01566888,
        -0.02521809,  0.02172849]])

and the same thing in Julia:

In [1]:
using LinearAlgebra
using Graphs
datadir = "C:/users/p/Datasets/"

"C:/users/p/Datasets/"

In [2]:
function Hope(g, sim, dim; beta=0.01, alpha=0.5)
    dim = dim*2
    A = Matrix(adjacency_matrix(g))
    n = nv(g)
    ## Katz
    if sim == :katz
        M_g = I - beta * A
        M_l = beta * A
    end
    ## Adamic-Adar
    if sim == :aa
        M_g = I
        D = diagm((x -> x > 1 ? 1/log(x) : 0.0).(g.degree()))
        M_l = A*D*A
        M_l[diagind(M_l)] .= 0.0
    end
    ## Common neighbors
    if sim == :cn
        M_g = I
        M_l = A*A
    end
    ## personalized page rank
    if sim == :ppr
        P = mapslices(A, dims=1) do x
            s = sum(x)
            iszero(s) ? fill(1/n, n) : x / s
        end
        M_g = I-alpha*P
        M_l = (1-alpha)*I
    end
    S = M_g \ M_l
    k = div(dim, 2)
    u, s, vt = svd(S)
    X1 = u[:, 1:k] * diagm(sqrt.(s[1:k]))
    ## undirected graphs have identical source and target embeddings
    if !is_directed(g)
        return X1
    else
        X2 = vtu[:, 1:k] * diagm(sqrt.(s[1:k]))
        return [X1 X2]
    end
end

g = SimpleGraphFromIterator([Graphs.SimpleEdge(parse.(Int,e[1]),parse.(Int,e[2])) 
                for e in split.(readlines(datadir * "ABCD/abcd_1000.dat"))])


{1000, 8327} undirected simple Int64 graph

In [3]:
@time Hope(g, :ppr, 16);

  3.745246 seconds (9.21 M allocations: 547.290 MiB, 6.52% gc time)


In [4]:
@time Hope(g, :ppr, 16);

  1.084728 seconds (8.56 k allocations: 93.181 MiB, 14.27% gc time)


As we can see there are some differences in performance of both languages. To understand how they differ we must introduce some basic elements of [Julia languange](https://julialang.org/).

# Control flow

## Conditional Evaluation

In [ ]:
if x < y
    println("x is less than y")
elseif x > y
    println("x is greater than y")
else
    println("x is equal to y")
end

For convienience sake we could replace <tt>if</tt> syntax ith logical operators <tt>&&</tt> (<tt>AND</tt>) and <tt>||</tt> (<tt>OR</tt>):

In [2]:
x = 4
x > 2 && println(x^2)

16


is the same as:

In [3]:
if (x>2)
    println(x^2)
end

16


same <tt>||</tt> is the same <tt>if not</tt> expression (note that in Julia exclamation mark <tt>!</tt> is negation symbol):

In [5]:
x < 3 || println(x/2)

2.0


In [6]:
if !(x<3)
    println(x/2)
end

2.0


<tt>if-else</tt> could be replaced by so called ternary operator <tt>?:</tt>:

In [10]:
x = 2
x ≥ 0 ? print(√x) : print("x smaller than 0!")

1.4142135623730951

## Compound Expressions

Compound expression <tt>begin</tt> is a convinient way of evaluating several subexpressions in order, returning the value of the last subexpression as its value:

In [11]:
z = begin 
    x = 1
    y = 2
    x + y
end

3

In [12]:
z

3

In [13]:
z = (x = 1; y = 2; x + y)
z

3

<tt>let</tt> expression creates a [local scope](https://docs.julialang.org/en/v1/manual/variables-and-scoping/) of variables and evaluate them inside it:

In [14]:
let 
    q = 1
    w = 2
    print(w + q)
end

3

In [15]:
q

LoadError: UndefVarError: q not defined

# Loops

Obviously, Julia supports loops:

In [16]:
i = 0
while i < 5
    println(i)
    i += 1
end

0
1
2
3
4


In [17]:
for i = 1:5
    println(i)
end

1
2
3
4
5


Julia also supports list comprehensions. However, unlike Python, list comprehensions are not improving the efficiency of code.
In some cases the way how they allocate resources might even decrease the efficiency of a code:

In [21]:
x = []
@time for i ∈ 1:1000
    push!(x,i^2)
end

  0.000161 seconds (987 allocations: 31.594 KiB)


In [20]:
@time x = [i^2 for i ∈ 1:1000];

  0.042625 seconds (45.17 k allocations: 2.636 MiB, 90.42% compilation time)


While working with loops, we must also about the scope of the variables. Julia is trying to compile as much code as it is possible, so using the same variables in different scopes is problematic. Basically Julia cannot assign the specific type to the variable, and therefore code cannot be optimized for this specific kind of variable:

In [29]:
x = 3
@time for i = 1:100000
    y = rand()
    x/y
end

  0.005092 seconds (200.00 k allocations: 3.052 MiB)


In [30]:
x = 3
@time for i = 1:100000
    global y = rand()
    x/y
end

  0.012618 seconds (200.00 k allocations: 3.052 MiB)
